In [160]:
import cupy as cp

In [161]:
import numpy as np

In [162]:
import sympy as sp

In [163]:
z = sp.IndexedBase('z')

In [164]:
x1, x2, theta, phi, t, = [z[i] for i in range(5)]
x = sp.Matrix([x1, x2, theta, phi])

In [165]:
l = 0.1
u1 = sp.Float(3.0)
u2 = 0.0

In [166]:
sp_transition = sp.Matrix(
    [
        sp.cos(theta) * sp.cos(phi) * u1,
        sp.sin(theta) * sp.cos(phi) * u1,
        sp.sin(phi) * u1 / l,
        u2,
    ]
)

In [167]:
f = sp.lambdify(z, sp_transition.tolist(), 'cupy')

In [168]:
args = cp.zeros(5)

In [169]:
f(args)

[[array(3.)], [array(0.)], [array(0.)], [0.0]]

In [178]:
import sympy as sp
import cupy as cp

# --- 1) Your original symbolic matrix (example from your screenshots) ---
x1, x2, theta, phi, u1, u2, t = sp.symbols(r'x_1 x_2 theta phi u_1 u_2 t')
l = 0.1

sp_transition = sp.Matrix([
    sp.cos(theta) * sp.cos(phi) * u1,
    sp.sin(theta) * sp.cos(phi) * u1,
    sp.sin(phi)   * u1 / l,
    u2,
    u1,
    u2,
])

# --- 2) Vectorize inputs: make the matrix depend on one vector x and scalar t ---
x = sp.IndexedBase('x')  # single vector argument
subs = {x1: x[0], x2: x[1], theta: x[2], phi: x[3], u1: x[4], u2: x[5]}
sp_transition_vecin = sp_transition.xreplace(subs)

# Compile to a CuPy function that takes (x, t)
F_raw = sp.lambdify((x, t), sp_transition_vecin, modules='cupy')

# --- 3) Nice Python wrapper: one CuPy vector in -> one CuPy vector out ---
def transition(x_vec: cp.ndarray, t_val: float) -> cp.ndarray:
    """
    x_vec : CuPy array with shape (6,) or (6,1)
    t_val : Python float (or NumPy/CuPy scalar)
    returns: CuPy array with shape (6,)
    """
    y = F_raw(x_vec, t_val)         # SymPy emits a CuPy array (typically shape (6, 1))
    return cp.asarray(y).reshape(-1)

# --- Example usage ---
x0 = cp.asarray([0.0, 0.0, 0.0, 0.0, 3.0, 0.0], dtype=cp.float64)
y0 = transition(x0, 0.0)            # y0 is a cupy.ndarray
# y0.shape == (6,)
y0

array([3., 0., 0., 0., 3., 0.])